# HAR LSTM training 

In [1]:
# HAR classification 
# Author: Burak Himmetoglu
# 8/15/2017

import pandas as pd 
import numpy as np
import os

def read_data(data_path, split = "train"):
	""" Read data """

	# Fixed params
	n_class = 6
	n_steps = 128

	# Paths
	path_ = os.path.join(data_path, split)
	path_signals = os.path.join(path_, "Inertial_Signals")

	# Read labels and one-hot encode
	label_path = os.path.join(path_, "y_" + split + ".txt")
	labels = pd.read_csv(label_path, header = None)

	# Read time-series data
	channel_files = os.listdir(path_signals)
	channel_files.sort()
	n_channels = len(channel_files)
	posix = len(split) + 5

	# Initiate array
	list_of_channels = []
	X = np.zeros((len(labels), n_steps, n_channels))
	i_ch = 0
	for fil_ch in channel_files:
		channel_name = fil_ch[:-posix]
		dat_ = pd.read_csv(os.path.join(path_signals,fil_ch), delim_whitespace = True, header = None)
		X[:,:,i_ch] = dat_.as_matrix()

		# Record names
		list_of_channels.append(channel_name)

		# iterate
		i_ch += 1

	# Return 
	return X, labels[0].values, list_of_channels

# How to normalize the input data correctly
def standardize(train, test):
	""" Standardize data """
	# Standardize train and test
	X_train = (train - np.mean(train, axis=0)[None,:,:]) / np.std(train, axis=0)[None,:,:]
	X_test = (test - np.mean(test, axis=0)[None,:,:]) / np.std(test, axis=0)[None,:,:]
	return X_train, X_test

# How to onehot-encoding
# def one_hot(labels, n_class = 6):
# 	""" One-hot encoding """
# 	expansion = np.eye(n_class)
# 	y = expansion[:, labels-1].T
# 	assert y.shape[1] == n_class, "Wrong number of labels!"
# 	return y

def get_batches(X, y, batch_size = 100):
	""" Return a generator for batches """
	n_batches = len(X) // batch_size
	X, y = X[:n_batches*batch_size], y[:n_batches*batch_size]

	# Loop over batches and yield
	for b in range(0, len(X), batch_size):
		yield X[b:b+batch_size], y[b:b+batch_size]

## Prepare data: Traing/Test data

In [2]:
Xtrain, Ytrain, list_ch_train = read_data(data_path="/home/arasdar/datasets/har-data/", split="train") # train
Xtest, Ytest, list_ch_test = read_data(data_path="/home/arasdar/datasets/har-data/", split="test") # test

assert list_ch_train == list_ch_test, "Mistmatch in channels!"

/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [3]:
Xtrain, Xtest = standardize(test=Xtest, train=Xtrain)

# Train/Validation Split

In [4]:
from sklearn.model_selection import train_test_split

Xtrain, Xvalid, Ytrain, Yvalid = train_test_split(Xtrain, Ytrain, stratify = Ytrain, random_state = 123)

In [5]:
print(Xtrain.shape, Xtrain.dtype)
print(Xvalid.shape, Xvalid.dtype)
print(Xtest.shape, Xtest.dtype)
print(Ytrain.shape, Ytrain.dtype, Ytrain.max(), Ytrain.min())
print(Yvalid.shape, Yvalid.dtype, Yvalid.max(), Yvalid.min())
print(Ytest.shape, Ytest.dtype, Ytest.max(), Ytest.min())

(5514, 128, 9) float64
(1838, 128, 9) float64
(2947, 128, 9) float64
(5514,) int64 6 1
(1838,) int64 6 1
(2947,) int64 6 1


### Hyperparameters

In [6]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.8.0
Default GPU Device: /device:GPU:0


In [7]:
# Hyperparameters
lstm_size = 27         # 3 times the amount of channels
lstm_layers = 1        # Number of layers
batch_size = 600       # Batch size
seq_len = 128          # Number of steps
learning_rate = 0.001  # Learning rate (default is 0.001)
epochs = 1000

# Fixed
n_classes = 6
n_channels = 9

### Construct the graph
Placeholders

In [8]:
# graph = tf.Graph()

# Construct placeholders
# with graph.as_default():
inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
indices_ = tf.placeholder(tf.int32, [None], name = 'indices')
# keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
# learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Construct inputs to LSTM

In [9]:
# with graph.as_default():
# Construct the LSTM inputs and LSTM cells
print(inputs_.shape)
lstm_in = tf.transpose(inputs_, [1,0,2]) # reshape into (seq_len, N, channels)
print(lstm_in.shape)
lstm_in = tf.reshape(lstm_in, [-1, n_channels]) # Now (seq_len*N, n_channels)
print(lstm_in.shape)

# Open up the tensor into a list of seq_len pieces
lstm_in = tf.split(lstm_in, seq_len, 0)
print(len(lstm_in), lstm_in[0].shape)

# Add LSTM layers
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
#     drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob_)
cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
initial_state = cell.zero_state(batch_size, tf.float32)

(?, 128, 9)
(128, ?, 9)
(?, 9)
128 (?, 9)


Define forward pass, cost function and optimizer:

In [10]:
# with graph.as_default():
# with tf.variable_scope('RNN', reuse=True):
outputs, final_state = tf.contrib.rnn.static_rnn(cell, lstm_in, dtype=tf.float32, initial_state = initial_state)

# We only need the last output tensor to pass into a classifier
logits = tf.layers.dense(outputs[-1], n_classes, name='logits')
labels = tf.one_hot(depth=n_classes, indices=indices_)

# Loss/cost using labels and logits
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))

# Accuracy using logits and labels
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

# Optimize using loss
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) # No grad clipping

In [11]:
print(len(outputs), outputs[0].shape)

128 (600, 27)


In [12]:
print(cost)

Tensor("Mean:0", shape=(), dtype=float32)


In [13]:
print(logits)

Tensor("logits/BiasAdd:0", shape=(600, 6), dtype=float32)


### Train the network

In [14]:
if (os.path.exists('checkpoints') == False):
    !mkdir checkpoints

In [ ]:
valid_acc, valid_loss = [], []
train_acc, train_loss = [], []

# with graph.as_default():
saver = tf.train.Saver()

with tf.Session() as sess:
    # Initialize the global variable instead of loading them or if there is nothing to load.
    sess.run(tf.global_variables_initializer())
    
    #     # Restore
    #     saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    #     saver.restore(sess,"checkpoints/har-lstm.ckpt")
    
    # Epochs/episodes of training/updating the model/network
    for e in range(epochs):
        
        # Training: Loop over batches
        state = sess.run(initial_state)
        acc_batch, loss_batch = [], []
        for Xbatch, Ybatch in get_batches(Xtrain, Ytrain, batch_size):

            # Feed dictionary
            feed_dict = {inputs_: Xbatch, indices_: Ybatch, initial_state: state}
            loss, _, state, acc = sess.run([cost, optimizer, final_state, accuracy], feed_dict)
            acc_batch.append(acc)
            loss_batch.append(loss)

        # Print at each epoch/iteration
        print("Training Epoch: {}/{}".format(e+1, epochs),
              "loss:{:.4f}".format(np.mean(loss_batch)),
              "acc:{:.4f}".format(np.mean(acc_batch)))

        # Store at each epoch/iteration
        train_loss.append(np.mean(loss_batch))
        train_acc.append(np.mean(acc_batch))
        
        # Validation: Loop over batches
        state = sess.run(initial_state)
        acc_batch, loss_batch = [], []
        for Xbatch, Ybatch in get_batches(Xvalid, Yvalid, batch_size):

            # Feed dictionary
            feed_dict = {inputs_: Xbatch, indices_: Ybatch, initial_state: state}
            loss, state, acc = sess.run([cost, final_state, accuracy], feed_dict)
            acc_batch.append(acc)
            loss_batch.append(loss)

        # Print at each epoch/iteration
        print("Validation Epoch: {}/{}".format(e+1, epochs),
              "loss:{:.4f}".format(np.mean(loss_batch)),
              "acc:{:.4f}".format(np.mean(acc_batch)))

        # Store at each epoch/iteration
        valid_loss.append(np.mean(loss_batch))
        valid_acc.append(np.mean(acc_batch))
            
    saver.save(sess,"checkpoints/model.ckpt")

Training Epoch: 1/1000 loss:1.4380 acc:0.2546
Validation Epoch: 1/1000 loss:1.3998 acc:0.2939
Training Epoch: 2/1000 loss:1.3664 acc:0.3263
Validation Epoch: 2/1000 loss:1.3278 acc:0.3772
Training Epoch: 3/1000 loss:1.2912 acc:0.4050
Validation Epoch: 3/1000 loss:1.2518 acc:0.4150
Training Epoch: 4/1000 loss:1.2130 acc:0.4217
Validation Epoch: 4/1000 loss:1.1717 acc:0.4306
Training Epoch: 5/1000 loss:1.1282 acc:0.4517
Validation Epoch: 5/1000 loss:1.0818 acc:0.4806
Training Epoch: 6/1000 loss:1.0427 acc:0.4996
Validation Epoch: 6/1000 loss:1.0068 acc:0.5056
Training Epoch: 7/1000 loss:0.9608 acc:0.5311
Validation Epoch: 7/1000 loss:0.9202 acc:0.5350
Training Epoch: 8/1000 loss:0.8825 acc:0.5604
Validation Epoch: 8/1000 loss:0.8374 acc:0.5744
Training Epoch: 9/1000 loss:0.8036 acc:0.6083
Validation Epoch: 9/1000 loss:0.7581 acc:0.6256
Training Epoch: 10/1000 loss:0.7307 acc:0.6298
Validation Epoch: 10/1000 loss:0.6919 acc:0.6100
Training Epoch: 11/1000 loss:0.6651 acc:0.6191
Validation 

Training Epoch: 87/1000 loss:0.1619 acc:0.7630
Validation Epoch: 87/1000 loss:0.1597 acc:0.7594
Training Epoch: 88/1000 loss:0.1528 acc:0.7646
Validation Epoch: 88/1000 loss:0.1604 acc:0.7561
Training Epoch: 89/1000 loss:0.1489 acc:0.7670
Validation Epoch: 89/1000 loss:0.1572 acc:0.7606
Training Epoch: 90/1000 loss:0.1481 acc:0.7672
Validation Epoch: 90/1000 loss:0.1570 acc:0.7606
Training Epoch: 91/1000 loss:0.1448 acc:0.7678
Validation Epoch: 91/1000 loss:0.1553 acc:0.7583
Training Epoch: 92/1000 loss:0.1465 acc:0.7665
Validation Epoch: 92/1000 loss:0.1547 acc:0.7567
Training Epoch: 93/1000 loss:0.1449 acc:0.7661
Validation Epoch: 93/1000 loss:0.1550 acc:0.7572
Training Epoch: 94/1000 loss:0.1452 acc:0.7663
Validation Epoch: 94/1000 loss:0.1534 acc:0.7611
Training Epoch: 95/1000 loss:0.1428 acc:0.7683
Validation Epoch: 95/1000 loss:0.1524 acc:0.7606
Training Epoch: 96/1000 loss:0.1428 acc:0.7680
Validation Epoch: 96/1000 loss:0.1539 acc:0.7600
Training Epoch: 97/1000 loss:0.1454 acc:

Training Epoch: 171/1000 loss:0.1466 acc:0.7650
Validation Epoch: 171/1000 loss:0.1528 acc:0.7594
Training Epoch: 172/1000 loss:0.1472 acc:0.7644
Validation Epoch: 172/1000 loss:0.1456 acc:0.7622
Training Epoch: 173/1000 loss:0.1460 acc:0.7646
Validation Epoch: 173/1000 loss:0.1534 acc:0.7611
Training Epoch: 174/1000 loss:0.1432 acc:0.7676
Validation Epoch: 174/1000 loss:0.1529 acc:0.7628
Training Epoch: 175/1000 loss:0.1450 acc:0.7663
Validation Epoch: 175/1000 loss:0.1562 acc:0.7611
Training Epoch: 176/1000 loss:0.1500 acc:0.7648
Validation Epoch: 176/1000 loss:0.1568 acc:0.7617
Training Epoch: 177/1000 loss:0.1520 acc:0.7644
Validation Epoch: 177/1000 loss:0.1580 acc:0.7622
Training Epoch: 178/1000 loss:0.1513 acc:0.7656
Validation Epoch: 178/1000 loss:0.1569 acc:0.7628
Training Epoch: 179/1000 loss:0.1493 acc:0.7644
Validation Epoch: 179/1000 loss:0.1525 acc:0.7622
Training Epoch: 180/1000 loss:0.1471 acc:0.7656
Validation Epoch: 180/1000 loss:0.1514 acc:0.7622
Training Epoch: 181/

Training Epoch: 255/1000 loss:0.1388 acc:0.7674
Validation Epoch: 255/1000 loss:0.1442 acc:0.7644
Training Epoch: 256/1000 loss:0.1424 acc:0.7669
Validation Epoch: 256/1000 loss:0.1403 acc:0.7656
Training Epoch: 257/1000 loss:0.1404 acc:0.7672
Validation Epoch: 257/1000 loss:0.1421 acc:0.7650
Training Epoch: 258/1000 loss:0.1390 acc:0.7667
Validation Epoch: 258/1000 loss:0.1413 acc:0.7633
Training Epoch: 259/1000 loss:0.1363 acc:0.7681
Validation Epoch: 259/1000 loss:0.1387 acc:0.7639
Training Epoch: 260/1000 loss:0.1352 acc:0.7681
Validation Epoch: 260/1000 loss:0.1359 acc:0.7656
Training Epoch: 261/1000 loss:0.1345 acc:0.7680
Validation Epoch: 261/1000 loss:0.1350 acc:0.7656
Training Epoch: 262/1000 loss:0.1326 acc:0.7685
Validation Epoch: 262/1000 loss:0.1336 acc:0.7650
Training Epoch: 263/1000 loss:0.1322 acc:0.7670
Validation Epoch: 263/1000 loss:0.1327 acc:0.7644
Training Epoch: 264/1000 loss:0.1341 acc:0.7633
Validation Epoch: 264/1000 loss:0.1423 acc:0.7561
Training Epoch: 265/

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Plot training and test loss
plt.figure(figsize = (6,6))
plt.plot(np.array(train_loss), 'r-', np.array(valid_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.plot(np.array(train_acc), 'r-', valid_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
loss, acc = [], []

with tf.Session() as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))

    # Testing: Loop over batches
    state = sess.run(initial_state)
    acc_batch, loss_batch = [], []
    for Xbatch, Ybatch in get_batches(Xtest, Ytest, batch_size):

        # Feed dictionary
        feed_dict = {inputs_: Xbatch, indices_: Ybatch, initial_state: state}
        loss, state, acc = sess.run([cost, final_state, accuracy], feed_dict)
        acc_batch.append(acc)
        loss_batch.append(loss)

    # Print at each epoch/iteration
    print("Epoch: {}/{}".format(e+1, epochs),
          "Test loss: {}".format(np.mean(loss_batch)),
          "Test acc: {}".format(np.mean(acc_batch)))